In [38]:
import pandas as pd
import pickle
import numpy as np

In [39]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data


In [40]:
df = pd.read_parquet('./yellow_tripdata_2022-01.parquet')

In [41]:
df.columns.size

19

Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data


In [42]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [43]:
df["duration"].describe()["std"]

46.44530513776802

Question 3: Fraction of the records left after dropping the outliers

In [44]:
df_without_outliers = df[(df.duration >= 1) & (df.duration <= 60)]

In [45]:
fraction_of_records_left = df_without_outliers["duration"].size/df["duration"].size
fraction_of_records_left

0.9827547930522406

Question 4: Dimensionality after OHE    

In [46]:
df = df_without_outliers

categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [47]:
X_train.shape

(2421440, 515)

Question 5: RMSE on train

In [48]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

# Calculate the RMSE

rmse = mean_squared_error(y_train, y_pred, squared=False)
print("RMSE:", rmse)


RMSE: 6.986191065500608


Question 6: RMSE on validation

In [49]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [50]:
df_train = read_dataframe('./yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./yellow_tripdata_2022-02.parquet')

In [51]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [52]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [53]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_val = lr.predict(X_val)

In [54]:
# Calculate the RMSE on val 

mean_squared_error(y_val, y_pred_val, squared=False)


7.795498122351478